```sql
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.
```

# **INTRODUCTIONS**

---

The source code is separated into multiple sections:

1. **[Tables](#tables)**
2. **[Stored Procedures](#stored-procedures)**
3. **[Views](#views)**
4. **[Datatino Configurations](#datatino-configurations)**

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/utils/conversions.ipynb",
        "src/utils/schemas.ipynb",
        "src/utils/protos.ipynb"
    ]
}
```

# **TABLES**
---

### STAGINGS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO


-- 1) CREATE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSSTAGE].[RIVM_SEWER_MEASUREMENTS_NL_2023]'))
BEGIN
    CREATE TABLE [VWSSTAGE].[RIVM_SEWER_MEASUREMENTS_NL_2023](
        [ID] [INT] PRIMARY KEY IDENTITY(1,1),
        [DATE_LAST_INSERTED] [DATETIME] NOT NULL DEFAULT GETDATE(),
        [VERSION] [VARCHAR](256) NULL,
        [DATE_OF_REPORT] [VARCHAR](256) NULL,
        [DATE_MEASUREMENT] [VARCHAR](256) NULL,
        [RNA_FLOW_PER_100000] [VARCHAR](256) NULL
        );
END


-- 2) CREATE INDEX(ES).....
DROP INDEX IF EXISTS [NCIX_RIVM_SEWER_MEASUREMENTS_NL_2023]
    ON [VWSSTAGE].[RIVM_SEWER_MEASUREMENTS_NL_2023]
GO

CREATE NONCLUSTERED INDEX [NCIX_RIVM_SEWER_MEASUREMENTS_NL_2023] 
ON [VWSSTAGE].[RIVM_SEWER_MEASUREMENTS_NL_2023] (
    [DATE_LAST_INSERTED] ASC
) WITH (STATISTICS_NORECOMPUTE = OFF, DROP_EXISTING = OFF, ONLINE = OFF, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
GO

### INTERMEDIATES

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO


-- 1) CREATE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSINTER].[RIVM_SEWER_MEASUREMENTS_NL_2023]'))
BEGIN
    CREATE TABLE [VWSINTER].[RIVM_SEWER_MEASUREMENTS_NL_2023](
        [ID] [INT] PRIMARY KEY IDENTITY(1,1),
        [DATE_LAST_INSERTED] [DATETIME] NOT NULL DEFAULT GETDATE(),
        [VERSION] [INT] NULL,
        [DATE_OF_REPORT] [DATETIME] NULL,
        [DATE_MEASUREMENT] [DATETIME] NULL,
        [RNA_FLOW_PER_100000] [BIGINT] NULL
        );
END


-- 2) CREATE INDEX(ES).....
DROP INDEX IF EXISTS [NCIX_RIVM_SEWER_MEASUREMENTS_NL_2023]
    ON [VWSINTER].[RIVM_SEWER_MEASUREMENTS_NL_2023]
GO

CREATE NONCLUSTERED INDEX [NCIX_RIVM_SEWER_MEASUREMENTS_NL_2023] 
ON [VWSINTER].[RIVM_SEWER_MEASUREMENTS_NL_2023] (
    [DATE_LAST_INSERTED] ASC
) WITH (STATISTICS_NORECOMPUTE = OFF, DROP_EXISTING = OFF, ONLINE = OFF, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
GO

### DESTINATIONS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO


-- 1) CREATE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSDEST].[RIVM_SEWER_MEASUREMENTS_GM_2023]'))
BEGIN
    CREATE TABLE [VWSDEST].[RIVM_SEWER_MEASUREMENTS_GM_2023](
        [ID] [INT] PRIMARY KEY IDENTITY(1,1),
        [DATE_LAST_INSERTED] [DATETIME] NOT NULL DEFAULT GETDATE(),
        [GMCODE] [VARCHAR](256) NULL,
        [DATE_START_UNIX] [BIGINT] NULL,
        [DATE_END_UNIX] [BIGINT] NULL,
        [AVERAGE] [INT] NULL
        );
END


-- 2) CREATE INDEX(ES).....
DROP INDEX IF EXISTS [NCIX_RIVM_SEWER_MEASUREMENTS_GM_2023]
    ON [VWSDEST].[RIVM_SEWER_MEASUREMENTS_GM_2023]
GO

CREATE NONCLUSTERED INDEX [NCIX_RIVM_SEWER_MEASUREMENTS_GM_2023] 
ON [VWSDEST].[RIVM_SEWER_MEASUREMENTS_GM_2023] (
    [DATE_LAST_INSERTED] ASC
) WITH (STATISTICS_NORECOMPUTE = OFF, DROP_EXISTING = OFF, ONLINE = OFF, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
GO

# **STORED PROCEDURES**
---

### STAGING &rarr; INTERMEDIATE MAPPING

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport. 
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.

-- 1) CREATE STORE PROCEDURE(S) STAGE -> INTER.....
CREATE OR ALTER PROCEDURE [DBO].[SP_RIVM_SEWER_MEASUREMENTS_GM_2023_STAGE_TO_INTER]
AS
BEGIN
    INSERT INTO [VWSINTER].[RIVM_SEWER_MEASUREMENTS_GM_2023] (
        [VERSION],
        [DATE_OF_REPORT],
        [YEAR],
        [WEEK],
        [START_DATE],
        [END_DATE],
        [REGION_CODE],
        [REGION_NAME],
        [RNA_FLOW_PER_100000_WEEKLYMEAN]
    )
    SELECT
        -- Provided datetime format in source file is yyyy-mm-dd
        [VERSION],
        CONVERT(DATETIME, [DATE_OF_REPORT], 102),
        [YEAR],
        [WEEK],
        CONVERT(DATETIME, [START_DATE], 102),
        CONVERT(DATETIME, [END_DATE], 102),
        [REGION_CODE],
        [REGION_NAME],
        IIF([RNA_FLOW_PER_100000_WEEKLYMEAN] IN ('', 'NA'), NULL, [RNA_FLOW_PER_100000_WEEKLYMEAN])
    FROM 
        [VWSSTAGE].[RIVM_SEWER_MEASUREMENTS_GM_2023]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[RIVM_SEWER_MEASUREMENTS_GM_2023])
END;
GO


### INTERMEDIATE &rarr; DESTINATION MAPPING

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

-- 1) CREATE STORE PROCEDURE(S) INTER -> DEST.....
CREATE OR ALTER PROCEDURE [DBO].[SP_RIVM_SEWER_MEASUREMENTS_GM_2023_INTER_TO_DEST]
AS
BEGIN
    INSERT INTO [VWSDEST].[RIVM_SEWER_MEASUREMENTS_GM_2023] (
        [GMCODE],
        [DATE_START_UNIX],
        [DATE_END_UNIX],
        [AVERAGE]
    )
    SELECT
        [REGION_CODE],
        dbo.CONVERT_DATETIME_TO_UNIX([START_DATE]),
        dbo.CONVERT_DATETIME_TO_UNIX([END_DATE]),
        CAST(round(([RNA_FLOW_PER_100000_WEEKLYMEAN] / 1.0E+11),0) as INT)
    FROM [VWSINTER].[RIVM_SEWER_MEASUREMENTS_GM_2023]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSINTER].[RIVM_SEWER_MEASUREMENTS_GM_2023])
    AND [DATE_OF_REPORT] = (SELECT MAX([DATE_OF_REPORT]) FROM [VWSINTER].[RIVM_SEWER_MEASUREMENTS_GM_2023])
    AND [RNA_FLOW_PER_100000_WEEKLYMEAN] IS NOT NULL -- only take actual measurements
END
GO


# **VIEWS**
---

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport. 
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.

CREATE OR ALTER VIEW [VWSDEST].[V_RIVM_SEWER_MEASUREMENTS_GM_2023] AS
    SELECT
        [GMCODE],
        [DATE_START_UNIX],
        [DATE_END_UNIX],
        [AVERAGE],
        CASE WHEN dbo.WEEK_START(dbo.CONVERT_UNIX_TO_DATETIME([DATE_START_UNIX])) < dateadd(day, -14, dbo.WEEK_START(DATE_LAST_INSERTED)) THEN 'true' ELSE 'false' END AS [DATA_IS_OUTDATED],
        dbo.CONVERT_DATETIME_TO_UNIX([DATE_LAST_INSERTED]) AS [DATE_OF_INSERTION_UNIX]
    FROM [VWSDEST].[RIVM_SEWER_MEASUREMENTS_GM_2023]
    WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM [VWSDEST].[RIVM_SEWER_MEASUREMENTS_GM_2023]);
GO

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport. 
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.

CREATE OR ALTER VIEW [VWSDEST].[V_DIFFERENCE_RIVM_SEWER_MEASUREMENTS_GM_2023] AS

WITH CTE1 AS
(
	SELECT
		[GMCODE],
		[DATE_START_UNIX] AS [NEW_DATE_UNIX],
		LAG([DATE_START_UNIX], 1, NULL) OVER (PARTITION BY [GMCODE] ORDER BY [GMCODE], [DATE_START_UNIX]) AS [OLD_DATE_UNIX],
		[AVERAGE],
		LAG([AVERAGE], 1, NULL) OVER (PARTITION BY [GMCODE] ORDER BY [GMCODE], [DATE_START_UNIX]) AS [OLD_VALUE]
	FROM [VWSDEST].[RIVM_SEWER_MEASUREMENTS_GM_2023]
	WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM [VWSDEST].[RIVM_SEWER_MEASUREMENTS_GM_2023])
),
CTE2 AS 
(
	SELECT [GMCODE], MAX([NEW_DATE_UNIX]) AS MAX_DATE
	FROM CTE1
	GROUP BY [GMCODE]
)
SELECT
	CTE1.[GMCODE], 
	CTE1.[NEW_DATE_UNIX], 
	CTE1.[OLD_DATE_UNIX], 
	CTE1.[OLD_VALUE], 
	CTE1.[AVERAGE] - CTE1.[OLD_VALUE] AS [DIFFERENCE]
FROM CTE1
JOIN CTE2 ON CTE1.[GMCODE] = CTE2.[GMCODE] AND CTE1.[NEW_DATE_UNIX]= CTE2.MAX_DATE -- only take last values per municipality

GO


# **DATATINO CONFIGURATIONS**
---

### WORKFLOWS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- 1) UPSERT WORKFLOW(S).....
DECLARE @workflow_name NVARCHAR(50) = 'RIVM_SEWER_MEASUREMENTS_NL_2023',
        @workflow_id INT,
        @workflow_description VARCHAR(256),
        @is_active INT;

-- 1.1) SET ENVIRONMENTAL VARIABLES.....
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @workflow_id = NULL;

SELECT TOP(1)
    @workflow_id = workflows.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[WORKFLOWS] workflows
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[DATAFLOW_ID] = workflows.[DATAFLOW_ID] AND v_workflows.[ID] = workflows.[ID]
WHERE v_workflows.[NAME] = @workflow_name;

SET @workflow_description = CONCAT('WORKFLOW: ', @workflow_name);

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_WORKFLOW]
    @id = @workflow_id, 
    @workflow_name = @workflow_name,
    @description = @workflow_description,
    @schedule = '0 14 * * TUE,WED,FRI',
    @active = @is_active;

-- 2) UPSERT SOURCE(S).....
DECLARE @source NVARCHAR(256),
        @source_id INT,
        @source_name NVARCHAR(256),
        @source_description NVARCHAR(256),
        @target_name VARCHAR(256),
        @location_type VARCHAR(50),
        @security_profile VARCHAR(50);

-- 2.1) SET ENVIRONMENTAL VARIABLES.....
SET @source = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'https://data.rivm.nl/data/vws/covid-19/COVID-19_rioolwaterdata_landelijk.csv'
    WHEN 'acceptance' THEN 'https://data.rivm.nl/data/vws/covid-19/COVID-19_rioolwaterdata_landelijk.csv'
    ELSE 'datafiles/RIVM/COVID-19_rioolwaterdata_landelijk.csv'
END;

SET @location_type = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'Web'
    WHEN 'acceptance' THEN 'Web'
    ELSE 'AzureBlob'
END;

SET @security_profile = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'RIVM'
    WHEN 'acceptance' THEN 'RIVM'
    ELSE 'RIVM'
END;

-- 2.2) UPSERT TABLE SOURCE(S): STAGING......
SET @source_id = NULL;
SET @source_name = CONCAT('SOURCE_', @workflow_name);
SET @source_description = CONCAT('LOAD: ', @workflow_name);
SET @target_name = CONCAT('VWSSTAGE.', @workflow_name);

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = 'Version|Date_of_report|Date_measurement|RNA_flow_per_100000',
    @target_columns = 'VERSION|DATE_OF_REPORT|DATE_MEASUREMENT|RNA_FLOW_PER_100000',
    @target_name = @target_name,
    @source_type = 'CsvFile',
    @location_type = @location_type,
    @delimiter_type = 'SemiColon',
    @security_profile= @security_profile;

-- 2.3) UPSERT STORED PROCEDURE SOURCE(S): STAGE TO INTER......
SET @source_id = NULL;
SET @source = CONCAT('dbo.SP_', @workflow_name, '_STAGE_TO_INTER');
SET @source_name = CONCAT('SOURCE_SP_', @workflow_name, '_STAGE_TO_INTER');
SET @source_description = CONCAT('MAP: ', @workflow_name, ' FROM STAGE TO INTER');

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = null,
    @target_columns = null,
    @target_name = null,
    @source_type = 'StoredProcedure',
    @location_type = 'N/A',
    @delimiter_type = 'N/A',
    @security_profile= @security_profile;

-- 2.4) UPSERT STORED PROCEDURE SOURCE(S): INTER TO DEST.....
SET @source_id = NULL;
SET @source = CONCAT('dbo.SP_', @workflow_name, '_INTER_TO_DEST');
SET @source_name = CONCAT('SOURCE_SP_', @workflow_name, '_INTER_TO_DEST');
SET @source_description = CONCAT('MAP: ', @workflow_name, ' FROM INTER TO DEST');

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = null,
    @target_columns = null,
    @target_name = null,
    @source_type = 'StoredProcedure',
    @location_type = 'N/A',
    @delimiter_type = 'N/A',
    @security_profile= @security_profile;

-- 3) UPSERT PROCESS(ES).....
DECLARE @process_id INT,
        @process_name NVARCHAR(256),
        @process_description NVARCHAR(256),
        @process_source_name VARCHAR(256);

-- 3.1) UPSERT TABLE PROCESS(ES): STAGING......
SET @process_id = NULL;
SET @process_name = CONCAT('PROCESS_', @workflow_name);
SET @process_description = CONCAT('LOAD: ', @workflow_name);
SET @process_source_name = CONCAT('SOURCE_', @workflow_name);

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_name,
    @active = 1;

-- 3.2) UPSERT STORED PROCEDURE PROCESS(S): STAGE TO INTER......
SET @process_id = NULL;
SET @process_name = CONCAT('PROCESS_SP_', @workflow_name, '_STAGE_TO_INTER');
SET @process_description = CONCAT('MAP: ', @workflow_name, ' FROM STAGE TO INTER');
SET @process_source_name = CONCAT('SOURCE_SP_', @workflow_name, '_STAGE_TO_INTER');

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_name,
    @active = 1;

-- 3.3) UPSERT STORED PROCEDURE PROCESS(S): INTER TO DEST......
SET @process_id = NULL;
SET @process_name = CONCAT('PROCESS_SP_', @workflow_name, '_INTER_TO_DEST');
SET @process_description = CONCAT('MAP: ', @workflow_name, ' FROM INTER TO DEST');
SET @process_source_name = CONCAT('SOURCE_SP_', @workflow_name, '_INTER_TO_DEST');

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_name,
    @active = 1;

-- 4) UPSERT DEPENDENC(Y)(IES).....
DECLARE @dependency_id INT,
        @dependency_name NVARCHAR(256),
        @dependency_description NVARCHAR(256),
        @dependency_dataflow_name NVARCHAR(256),
        @dependency_process_name NVARCHAR(256);

-- 4.1) UPSERT STAGING TO STORE PROCEDURE DEPENDENC(Y)(IES).....
SET @dependency_id = NULL;
SET @dependency_name = CONCAT('DEPENDENCY_', @workflow_name,'_STAGE_TO_INTER');
SET @dependency_dataflow_name = CONCAT('PROCESS_SP_', @workflow_name, '_STAGE_TO_INTER');
SET @dependency_process_name = CONCAT('PROCESS_', @workflow_name);
SET @dependency_description = CONCAT('TRIGGER ', @dependency_dataflow_name,' AFTER ', @dependency_process_name ,' HAS FINISHED');

SELECT TOP(1)
    @dependency_id = dependencies.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[DEPENDENCIES] dependencies
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_DEPENDENCIES] v_dependencies ON v_dependencies.[ID] = dependencies.[ID]
WHERE dependencies.[NAME] = @dependency_name
    AND v_dependencies.[WORKFLOW_NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_DEPENDENCY]
    @id = @dependency_id,
    @dataflow_name = @dependency_dataflow_name,
    @dataflowtype_id = 2,
    @dependency_name = @dependency_process_name,
    @dependencytype_id = 2,
    @workflow_name = @workflow_name,
    @name = @dependency_name,
    @description = @dependency_description,
    @active = 1;

-- 4.2) UPSERT STORED PROCEDURE TO STORED PROCEDURE DEPENDENC(Y)(IES)......
SET @dependency_id = NULL;
SET @dependency_name = CONCAT('DEPENDENCY_', @workflow_name,'_INTER_TO_DEST');
SET @dependency_dataflow_name = CONCAT('PROCESS_SP_', @workflow_name, '_INTER_TO_DEST');
SET @dependency_process_name = CONCAT('PROCESS_SP_', @workflow_name, '_STAGE_TO_INTER');
SET @dependency_description = CONCAT('TRIGGER ', @dependency_dataflow_name,' AFTER ', @dependency_process_name ,' HAS FINISHED');

SELECT TOP(1)
    @dependency_id = dependencies.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[DEPENDENCIES] dependencies
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_DEPENDENCIES] v_dependencies ON v_dependencies.[ID] = dependencies.[ID]
WHERE dependencies.[NAME] = @dependency_name
    AND v_dependencies.[WORKFLOW_NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_DEPENDENCY]
    @id = @dependency_id,
    @dataflow_name = @dependency_dataflow_name,
    @dataflowtype_id = 2,
    @dependency_name = @dependency_process_name,
    @dependencytype_id = 2,
    @workflow_name = @workflow_name,
    @name = @dependency_name,
    @description = @dependency_description,
    @active = 1;
GO

### PROTOS

In [ ]:
-- 1) CREATE PROTOS CONFIGURATION(S).....
DECLARE @current_proto_name VARCHAR(50)

DECLARE @proto_name_table TABLE (
    [PROTO_NAME] VARCHAR(50)
);

-- DELETE ALL sewer-GM combo's
DELETE FROM [DATATINO_PROTO_1].[CONFIGURATIONS]
  WHERE NAME = 'sewer'
  AND PROTO_ID IN (SELECT Distinct [ID]
    FROM [DATATINO_PROTO_1].[PROTOS]
    WHERE NAME NOT LIKE 'GM_COLLECTION' AND NAME LIKE 'GM%' 
	)

-- 1) CREATE PROTO(S)...
INSERT INTO @proto_name_table 
SELECT Distinct [NAME]
    FROM [DATATINO_PROTO_1].[PROTOS]
    WHERE NAME NOT LIKE 'GM_COLLECTION' AND NAME LIKE 'GM%' 
ORDER BY NAME

DECLARE CUR CURSOR LOCAL FAST_FORWARD
FOR
SELECT [PROTO_NAME]
FROM @proto_name_table;

OPEN CUR;

FETCH NEXT FROM CUR
INTO @current_proto_name;

WHILE @@FETCH_STATUS = 0
BEGIN

-- 1) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_RIVM_SEWER_MEASUREMENTS_GM_2023',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'sewer',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = @current_proto_name,
        @constraint_key_name VARCHAR(50)= 'GMCODE',
        @grouped_key_name VARCHAR(50) = null,
        @grouped_last_update_name VARCHAR(50) = null,
        @proto_name VARCHAR(50) = @current_proto_name,
        @columns VARCHAR(256) = 'DATE_START_UNIX|DATE_END_UNIX|AVERAGE|DATA_IS_OUTDATED|DATE_OF_INSERTION_UNIX',
        @layout_type_id INT = 1, -- VALUES
        @last_update_name VARCHAR(50) = 'DATE_START_UNIX',
        @is_active INT;

-- 1.1) SET ENVIRONMENTAL VARIABLES.....
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT = null,
        @config_id BIGINT = null;

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

-- 2) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 3) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

FETCH NEXT FROM CUR
    INTO @current_proto_name;    
END

CLOSE CUR;
DEALLOCATE CUR;
DELETE FROM @proto_name_table;
GO

In [ ]:
-- 1) CREATE PROTOS CONFIGURATION(S).....
DECLARE @current_proto_name VARCHAR(50)

DECLARE @proto_name_table TABLE (
    [PROTO_NAME] VARCHAR(50)
);

-- DELETE ALL difference/sewer__average-GM combo's
DELETE FROM [DATATINO_PROTO_1].[CONFIGURATIONS]
  WHERE NAME = 'difference/sewer__average'
  AND PROTO_ID IN (SELECT Distinct [ID]
    FROM [DATATINO_PROTO_1].[PROTOS]
    WHERE NAME NOT LIKE 'GM_COLLECTION' AND NAME LIKE 'GM%' 
	)

-- 1) CREATE PROTO(S)...
INSERT INTO @proto_name_table 
SELECT Distinct [NAME]
    FROM [DATATINO_PROTO_1].[PROTOS]
    WHERE NAME NOT LIKE 'GM_COLLECTION' AND NAME LIKE 'GM%' 
ORDER BY NAME

DECLARE CUR CURSOR LOCAL FAST_FORWARD
FOR
SELECT [PROTO_NAME]
FROM @proto_name_table;

OPEN CUR;

FETCH NEXT FROM CUR
INTO @current_proto_name;

WHILE @@FETCH_STATUS = 0
BEGIN

-- 1) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_DIFFERENCE_RIVM_SEWER_MEASUREMENTS_GM_2023',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'difference/sewer__average',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = @current_proto_name,
        @constraint_key_name VARCHAR(50)= 'GMCODE',
        @grouped_key_name VARCHAR(50) = null,
        @grouped_last_update_name VARCHAR(50) = null,
        @proto_name VARCHAR(50) = @current_proto_name,
        @columns VARCHAR(256) = 'OLD_VALUE|DIFFERENCE|OLD_DATE_UNIX|NEW_DATE_UNIX',
        @layout_type_id INT = 3, -- LAST VALUE
        @last_update_name VARCHAR(50) = 'NEW_DATE_UNIX',
        @is_active INT;

-- 1.1) SET ENVIRONMENTAL VARIABLES.....
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT = null,
        @config_id BIGINT = null;

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

-- 2) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 3) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

FETCH NEXT FROM CUR
    INTO @current_proto_name;    
END

CLOSE CUR;
DEALLOCATE CUR;
DELETE FROM @proto_name_table;
GO

GM_COLLECTION

In [ ]:
-- DELETE ALL sewer-GM_COLLECTION combo's
DELETE FROM [DATATINO_PROTO_1].[CONFIGURATIONS]
  WHERE NAME = 'sewer'
  AND PROTO_ID IN (SELECT Distinct [ID]
    FROM [DATATINO_PROTO_1].[PROTOS]
    WHERE NAME = 'GM_COLLECTION'
	)

-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_RIVM_SEWER_MEASUREMENTS_GM_2023',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'sewer',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = null,
        @grouped_last_update_name VARCHAR(50) = null,
        @proto_name VARCHAR(50) = 'GM_COLLECTION',
        @columns VARCHAR(1024) = 'GMCODE|DATE_START_UNIX|DATE_END_UNIX|AVERAGE|DATA_IS_OUTDATED|DATE_OF_INSERTION_UNIX',
        @layout_type_id INT = 4, -- VALUESDIRECT
        @last_update_name VARCHAR(50) = 'DATE_START_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT = null,
        @config_id BIGINT = null;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;
    
SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO